In [1]:
#pip install tensorflow

In [2]:
import random
import tensorflow as tf
from tensorflow.keras.optimizers import SGD # from tensorflow.keras.optimizers.legacy import SGD # 
from pyvi import ViTokenizer, ViPosTagger, ViUtils
from tensorflow.python.keras.layers import Dense, Dropout # from keras.layers import Dense, Dropout
from tensorflow.python.keras.models import load_model # from keras.models import load_model
from tensorflow.python.keras.models import Sequential # from keras.models import Sequential
import numpy as np
import pickle, os
import json, warnings
import nltk
from nltk.stem import WordNetLemmatizer

warnings.filterwarnings("ignore")

In [3]:
lemmatizer = WordNetLemmatizer()
try:
    nltk.download('omw-1.4')
    nltk.download("punkt")
    nltk.download("wordnet")
except :
    print("need to check")

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [4]:
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
current_fpath = os.getcwd()
print("current file-path", current_fpath)
data_folder = '\\'.join(current_fpath.split('\\')[:-1]) # r"/workspaces/AI_chatbot_flask"
print(f"file in this directory: {os.listdir(data_folder)}")
data_file = open(f"{data_folder}\\json_data\\intents.json", encoding='utf-8').read()
intents = json.loads(data_file)

current file-path C:\Users\Admin\Desktop\jupyter_lab\chatbot\retrieval_based_bot\notebook
file in this directory: ['.ipynb_checkpoints', 'json_data', 'models', 'notebook']


In [5]:
# words
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # take each word and tokenize it
        patt = ViTokenizer.tokenize(pattern)
        w = nltk.word_tokenize(patt)
        words.extend(w)
        # adding documents
        documents.append((w, intent["tag"]))

        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])

In [6]:
# lemmatizer
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

#print(len(documents), "documents")
print(f"Number of pre-trained class {len(classes)}")
print(f"Number of words: {len(words)} \n The last 20 words {words[-21:-1]}")

pickle.dump(words, open(f"{data_folder}//models//words.pkl", "wb"))
pickle.dump(classes, open(f"{data_folder}//models//classes.pkl", "wb"))

Number of pre-trained class 3
Number of words: 47 
 The last 20 words ['là', 'lập_trình', 'mày', 'ni', 'ohayo', 'qui', 'salut', 'sayonara', 'thank', 'there', 'tschuss', 'tạm_biệt', 'vous', 'wer', 'who', 'xin', 'you', 'your', 'ê', 'êtes']


In [7]:
# training initializer
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

In [8]:
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training, dtype='object')
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")

Training data created


In [9]:
# actual training
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(512, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.4))
model.add(Dense(256, activation="relu"))
model.add(Dense(128, activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               24576     
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_2 (Dense)              (None, 128)               32896     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 189,187
Trainable params: 189,187
Non-trainable params: 0
__________________________________________________

In [10]:
from keras import callbacks 

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer='SGD', metrics=["accuracy"])

earlystopping = callbacks.EarlyStopping(monitor ="loss", mode ="min", patience = 5, restore_best_weights = True)
callbacks =[earlystopping]

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=100, batch_size=5, verbose=1, callbacks = callbacks)
model.save(f"{data_folder}//models//chatbot_model.h5", hist)
print("model created")

Epoch 1/100
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
7/7 [==============================] - 0s 1ms/step - loss: 1.1062 - accuracy: 0.3714
Epoch 2/100
7/7 [==============================] - 0s 830us/step - loss: 1.0801 - accuracy: 0.5143
Epoch 3/100
7/7 [==============================] - 0s 1ms/step - loss: 1.0797 - accuracy: 0.5143
Epoch 4/100
7/7 [==============================] - 0s 1ms/step - loss: 1.0658 - accur

7/7 [==============================] - 0s 1ms/step - loss: 0.5549 - accuracy: 0.7429
Epoch 76/100
7/7 [==============================] - 0s 1ms/step - loss: 0.5364 - accuracy: 0.8286
Epoch 77/100
7/7 [==============================] - 0s 853us/step - loss: 0.5337 - accuracy: 0.8571
Epoch 78/100
7/7 [==============================] - 0s 1ms/step - loss: 0.5225 - accuracy: 0.8286
Epoch 79/100
7/7 [==============================] - 0s 1ms/step - loss: 0.5068 - accuracy: 0.8571
Epoch 80/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4898 - accuracy: 0.8286
Epoch 81/100
7/7 [==============================] - 0s 2ms/step - loss: 0.4726 - accuracy: 0.8571
Epoch 82/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4778 - accuracy: 0.8571
Epoch 83/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4450 - accuracy: 0.8571
Epoch 84/100
7/7 [==============================] - 0s 1ms/step - loss: 0.4285 - accuracy: 0.8571
Epoch 85/100
7/7 [=============

In [14]:
model = load_model(f"{data_folder}//models//chatbot_model.h5")
intents = json.loads(open(f"{data_folder}//json_data//intents.json", encoding='utf-8').read())
words = pickle.load(open(f"{data_folder}//models//words.pkl", "rb"))
classes = pickle.load(open(f"{data_folder}//models//classes.pkl", "rb"))
data_file = open(f"{data_folder}//json_data//intents.json", encoding='utf-8').read()
intents = json.loads(data_file)

# chat functionalities
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(ViTokenizer.tokenize(sentence))
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

In [15]:
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)

In [16]:
def rounding_text_hour(text):

    mask1 = any(word in text for word in ['tiếng', 'phút', 'giờ', 'minute', 'minutes'])
    mask2 = any(word in text for word in ['giờ làm việc', 'giờ hành chính', 'giờ đi làm', 'giờ vào làm', 'mấy giờ', "giờ ngủ trưa",
                                          'giờ có mặt', 'giờ tan sở', 'giờ ra về', 'giờ giải lao', 'giờ nghỉ trưa', "giờ nghỉ ngơi"])
    if mask1 & ~mask2:
        text = text.replace('giờ', 'tiếng').replace('hour', 'tiếng').replace('minute', 'phút').replace('mins', 'phút')
        h = re.findall('[0-9]+tiếng', text.replace(" ", ""))
        m = re.findall('[0-9]+phút', text.replace(" ", ""))
        if ((len(h) > 0) & (len(m) > 0)):
            h = h[0].replace("tiếng", "")
            m = m[0].replace("phút", "")
            if (int(m) < 0) or (int(m) > 60):
                txt = "số giờ (hoặc phút) làm việc không hợp lệ"
            elif int(m) < 60:
                txt = f"dưới {int(h) + 1} tiếng nhưng đã trên {int(h)} giờ làm việc"
            else:
                txt = f"dưới {int(h) + (m // 60) + 1} tiếng nhưng đã trên {int(h)} giờ làm việc"
        elif (len(h) == 0):
            txt = "dưới 1 tiếng làm việc"
        elif (len(m) == 0):
            txt = f"dưới {re.findall('[0-9]', h[0])[0]} giờ làm việc"
        else: 
            txt = text
        return txt    
    else: 
        return text

In [18]:
while True:
    sentence = input("You: ")
    sentence = rounding_text_hour(sentence)
    if sentence == "quit":
        break
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.3
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    ints = return_list

    if len(return_list) > 0:
        tag = ints[0]["intent"]
        list_of_intents = intents["intents"]

        for i in list_of_intents:
            if i["tag"] == tag:
                result = random.choice(i["response"])
                print(f"BOT: {result}")
    else: 
        print("xin giải thích rõ ràng hơn vì có thể bot không hiểu hoặc chưa được học!")

You: hi there
BOT: (VI) chào bạn, bạn cần hỗ trợ điều gì?
(EN) Hi, what support do you need?
(GE) Hallo, welche Unterstützung brauchst du?
(FR) Bonjour, de quel soutien avez-vous besoin?
You: quit
